In [ ]:
import numpy as np
import os
import subprocess
import time
from ase.io.trajectory import Trajectory
from ase.io import read, write
from dataclasses import dataclass

In [ ]:
### Function
## Common
def copy_file(source, target):
    """
    Copy a file from the source path to the target path
    """
    copy = f'cp {source} {target}'
    run = subprocess.Popen(copy, shell=True)
    run.wait()
        
def Checkdir(keyword):
    '''
    Check whether the file with corresponding key already existed.
    When the file existed, it will be removed
    '''
    root = os.getcwd()
    dir_list = os.listdir("./")
    for path in dir_list:
        if keyword in path:
            dir_path = f"{root}/{path}"
            if os.path.exists(dir_path):
                print(f"The dictionary {dir_path} will be removed!")
                shutil.rmtree(dir_path)
    
    
def showtime(func):
    def wrapper(System):
        start_time = time.time()
        func(System)
        end_time = time.time()
        print('Running time is {}'.format(end_time - start_time))
    return wrapper


def check_value_type(value, value_type):
    if isinstance(value, value_type):
        pass
    else:
        print(f"The {value} has wrong data type, It must be {value_type}.")
    

def change_symbol(value):
    if "," in str(value):
        value_new = str(value).strip().replace(",", ".")
        return float(value_new)
    else:
        return float(str(value).strip())


def lattice(element):
    Al = {"a":[3.17, 3.19], "mass":26.982, "melting_point":600}
    Cu = {"a":[3.62, 3.72], "mass":63.546, "melting_point":1360}
    Au = {"a":[3.10, 4.10], "mass":196.97, "melting_point":2000}
    if str(element) in ["Al", "Cu", "Au"]:
        value_min, value_max = float(vars()[str(element)]['a'][0]), float(vars()[str(element)]['a'][1])
        lattice_constant = change_symbol(input(f"Please select the lattice constant between {value_min} and {value_max}"))
        if lattice_constant > value_min and lattice_constant < value_max:
            atom_mass = vars()[str(element)]["mass"]
        else:
            print("Please give the value in right range")
    else:
        print("Please write the right name of the element")
    melting_point = vars()[str(element)]["melting_point"]
    return lattice_constant, atom_mass, melting_point


## Lammps simulation
def copy_potential(System):
    copy_file(f"./potentials/{System.potential_name}", f"./{System.Project_name}/{System.potential_name}")

def write_input(System):
    """
    Problem: NPT or NVT
    """
    with open(f"./{System.Project_name}/lammps_input", "w+") as fw:
        fw.write(f"""
dimension 3
units metal
atom_style atomic
boundary p p p

lattice fcc {System.lattice_constant}
region box block 0 {System.box_length} 0 {System.box_length} 0 {System.box_length}
create_box 1 box
create_atoms 1 box
mass 1 {System.atom_mass}

pair_style eam/alloy
pair_coeff * * {System.potential_name} {System.element}

neighbor 3.0 bin

minimize 1e-8 1e-8 10000 100000
min_style cg

write_data original

change_box all x delta 0 3 
change_box all y delta 0 3 
change_box all z delta 0 3 boundary s s s

timestep {System.timestep}

thermo {System.thermo_time}

thermo_style custom step temp vol press

velocity all create {System.start_temperature} {System.random_number} mom yes rot yes dist gaussian

dump 1 all custom 1000 melting id type xs ys zs

fix 2 all nvt temp {System.start_temperature} {System.end_temperature} 0.1

run {System.running_steps}
        """)


#@showtime
def run_lammps(input_file):
    serial_run = 'mpirun -n 8 lmp_mpi <{} > log'.format(str(input_file))
    run = subprocess.Popen(serial_run, shell=True)
    run.wait()


#@showtime
def animate_show(particle_size=0.5):
    traj = read(filename=f"./melting", format="lammps-dump-text", index=":")
    write(f"./melting.traj", traj)
    import nglview
    traj = Trajectory("./melting.traj")
    movie = nglview.show_asetraj(traj)
    movie.display(gui=True, style='ngl')
    movie.add_cartoon(selection='all')
    movie.add_spacefill(radius_type="vdw", scale=0.5, radius=0.5, color='red', max_frame=500)
    movie.remove_ball_and_stick()
    movie.add_unitcell()
    movie.camera = "orthographic"
    return movie


# Input for some variable
def collect_input():
    Project_name = input("Please give the name of your project: ")
    element = input("Please selecet an element from Al, Cu or Au: ")
    lattice_constant, atom_mass, melting_point = lattice(element)
    
    box_length = input("Please give the length(integer) of box between 1 and 10: ")
    if int(box_length) > 1 and int(box_length) < 10:
        pass
    else:
        print("Please give the value between 10 and 15!")
    
    timestep = 0.001
    start_temperature = input(f"Please give the intial temperature in K (usually it can be room temprature): ")
    seed = input("Please give a seed for your MD")
    end_temperature = input(f"Please give the finally temperature in K (usually it can be higher than the melting point {melting_point}): ")
    return Project_name, element, lattice_constant, atom_mass, melting_point, box_length, timestep, start_temperature, seed, end_temperature

In [ ]:
# Dataclass
@dataclass
class System:
    Project_name: str
    element:str
    lattice_constant: float
    atom_mass: float
    melting_point: int
    box_length: int
    timestep: float
    start_temperature: int
    seed: int
    end_temperature: int
    thermo_time = 5000
    running_steps = 150000
    

    def __post_init__(self):
        self.keyword = self.Project_name
        if self.element == "Al":
            self.potential_name = "Al03.eam.alloy"
        if self.element == "Cu":
            self.potential_name = "Cu01.eam.alloy"
        if self.element == "Au":
            self.potential_name = "Au.eam"
        
        seed = np.int64(self.seed)
        np.random.seed(seed)
        self.random_number = int("".join([str(x) for x in np.random.randint(10, size=6)]))

In [ ]:
if __name__ == '__main__':
    Lammps_system = System(*collect_input())

In [ ]:
# Prepare Input file
os.mkdir(f"./{Lammps_system.Project_name}")
copy_potential(Lammps_system)
write_input(Lammps_system)

# Run Lammps
os.chdir(f"./{Lammps_system.Project_name}")
run_lammps("lammps_input")

In [ ]:
animate_show(particle_size=0.5)